In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import pickle
from PIL import Image



In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def carregar_pickle(nome_arquivo):
  with open(nome_arquivo, 'rb') as arquivo:
    objeto = pickle.load(arquivo)
  return objeto

In [3]:
model2Test = keras.models.load_model('exp/bmw6/best_model_acc6_03.h5', custom_objects={"f1_m": f1_m , 'precision_m': precision_m,
        'recall_m': recall_m })

In [4]:
raw_data_set = carregar_pickle('dataset_H3_W3S1.pkl')
input_full = raw_data_set['inputs']
output_full = raw_data_set['outputs']
sequences_full = raw_data_set['sequences']

In [69]:
def save_grayscale_image(matrix, filename):
    # Normaliza a matriz para o intervalo [0, 255]
    normalized_matrix = (matrix - np.min(matrix)) * (255.0 / (np.max(matrix) - np.min(matrix)))
    # Converte para tipo inteiro
    grayscale_image = Image.fromarray(normalized_matrix.astype(np.uint8), mode='L')
    grayscale_image.save(filename)

In [32]:
def getFirstNElementsWithPredictAboveOrUnderPredict(inputFull, model, isAboveOrequal, predictValueRef, n):
    sList = []
    index = 0
    while True:
        v = model.predict(inputFull[0+index:1+index])[0][0]

        print('v: ' + str(v))

        if isAboveOrequal and v >= predictValueRef:
            sList.append(index)
        elif not isAboveOrequal and v < predictValueRef:
            sList.append(index)

        if len(sList) >= n:
            break
        
        print('sList len: ' + str(len(sList)))
        
        index = index + 1

    return sList

def getInputFromInputFullAndIndex(inputFull, index):
    return inputFull[0+index:1+index]

In [33]:
goodList = getFirstNElementsWithPredictAboveOrUnderPredict(input_full, model2Test, True, 0.95, 5)

1/1 [==============================] - 0s 41ms/step
v: 0.07839887
sList len: 0
1/1 [==============================] - 0s 51ms/step
v: 0.003686926
sList len: 0
1/1 [==============================] - 0s 58ms/step
v: 0.0062189577
sList len: 0
1/1 [==============================] - 0s 56ms/step
v: 0.23412523
sList len: 0
1/1 [==============================] - 0s 62ms/step
v: 0.8609274
sList len: 0
1/1 [==============================] - 0s 51ms/step
v: 0.6608417
sList len: 0
1/1 [==============================] - 0s 60ms/step
v: 0.049053676
sList len: 0
1/1 [==============================] - 0s 59ms/step
v: 0.43479964
sList len: 0
1/1 [==============================] - 0s 58ms/step
v: 0.9274257
sList len: 0
1/1 [==============================] - 0s 41ms/step
v: 0.0030867956
sList len: 0
1/1 [==============================] - 0s 54ms/step
v: 0.76205665
sList len: 0
1/1 [==============================] - 0s 64ms/step
v: 0.98538387
sList len: 1
1/1 [==============================] - 0s 59ms/st

In [34]:
badList = getFirstNElementsWithPredictAboveOrUnderPredict(input_full, model2Test, False, 0.5, 5)

1/1 [==============================] - 0s 60ms/step
v: 0.07839887
sList len: 1
1/1 [==============================] - 0s 74ms/step
v: 0.003686926
sList len: 2
1/1 [==============================] - 0s 74ms/step
v: 0.0062189577
sList len: 3
1/1 [==============================] - 0s 64ms/step
v: 0.23412523
sList len: 4
1/1 [==============================] - 0s 55ms/step
v: 0.8609274
sList len: 4
1/1 [==============================] - 0s 70ms/step
v: 0.6608417
sList len: 4
1/1 [==============================] - 0s 54ms/step
v: 0.049053676


In [42]:
goodList

[11, 20, 21, 22, 23]

In [43]:
input = getInputFromInputFullAndIndex(input_full, 11)

In [107]:
sequence = sequences_full[0+11:1+11]
sequence

['TCGTCAGTTCGTACACCATTGGTGCCAGTGACTGTGGTCAATTCGGTAGAAGTAGAGGTAAAAGTGCTGTTCCATGGCTCAGTTGTAGTTATGATGGTGCTAGCAGTTGTTGGAGTTCTGATGACAATGACGGTTTCGTCAGTTGGAACGCCGTTGGTACCGGTGACGGTGGTCATTTCAGTAGATGTAGAAGTGAAAGTACCGGTCCATGGTTCCGTTGTAGTTATGGTAGTACTGACAGTATAATTTGAAGGGTCTGGAATGGTACAGTTTGGCTGGCTTAGATTGTTGTCAAAAGTATATACGTACCCTTCAAAGTCATCACTAACGGTAGTGCCATCTGGTAGTGTCACACTAATTGGAAGTGTACCCCAGGCAACGGCATTTGAGTAAACAATCTTCATTGGATAATAGAAACCAGCATACATGTAGACAGTCCCTGTAATATTATCAGGGGGACTTCCATTCCATGGCTTGATACCATTGATGGTGAAGTTAGT']

In [ ]:
model2Test.predict(input)

In [74]:
model2Test.layers

In [99]:
intermediate_output = model2Test.layers[6].output
get_output = keras.backend.function([model2Test.input], [intermediate_output])
output = get_output([input])[0]

In [100]:
#output_reshaped = output.reshape(output.shape[1], output.shape[2])
output_reshaped = output

In [101]:
output_reshaped.shape

(1, 60)

In [102]:
output_reshaped

array([[0.00809478, 0.0418728 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.09380104, 0.        ,
        0.        , 0.02054345, 0.        , 0.0180725 , 0.        ,
        0.1301553 , 0.        , 0.        , 0.        , 0.19605362,
        0.00779951, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.19382474, 0.        , 0.        ,
        0.        , 0.        , 0.01706999, 0.        , 0.        ,
        0.02224108, 0.        , 0.00826497, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.03742843,
        0.        , 0.        , 0.10345274, 0.        , 0.        ]],
      dtype=float32)

In [106]:
# exp/network_exploration/best_only6/goods
save_grayscale_image(input.reshape(input.shape[1], input.shape[2]), 'exp/network_exploration/best_only6/goods/input_00.png')